<a href="https://colab.research.google.com/github/derzhavin3016/CompMath/blob/master/Lab2/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя лабораторная работа №2 по вычислительной математике
Державин Андрей, Б01-909 группа


Задача II.10.6 (к)

$$
  \left\lbrace
  \begin{matrix}
    a_{11} x_1 + a_{12} x_2 + \dots + a_{1n} x_n &= &f_1 \\
    \dots  &\dots   &\dots \\
    a_{n1} x_1 + a_{n2} x_2 + \dots + a_{nn} x_n &= &f_n
  \end{matrix}
  \right. ,
$$
где
$$
n = 10, \: 
a_{ii} = 1, \: 
a_{ij} = \frac{1}{i + j} (i \neq j), \:
f_i = \frac{1}{i} 
$$

In [2]:
import numpy as np
from numpy import linalg as ling

Определяем константы: размер матрицы и погрешность (для сравнения чисел с 0)

In [60]:
SIZE = 10
ACCURACY = 1e-10

Функция сравнения числа с нулём с учётом погрешности

In [61]:
def is_zero(num: float):
  return abs(num) < ACCURACY

Функция нахождения срочки в матрице, в которой в столбце `idx` стоит ненулеове число

In [27]:
def find_non_zero(arr, col_idx):
  for i in range(col_idx + 1, SIZE):
    if not is_zero(arr[i][col_idx]):
      return col_idx
  return -1

Функция приведения к диагональному виду (прямой ход Гаусса)

In [39]:
def to_diag(matr):
  for i in range(SIZE - 1):
    if is_zero(matr[i][i]):
      non_z_line = find_non_zero(matr, i)
      if non_z_line == -1:
        raise ValueError("Invalid Matrix")
      # swap rows
      matr[[i, non_z_line]] = matr[[non_z_line, i]]

    div = matr[i][i]
    for j in range(i + 1, SIZE):
      matr[j] += (-matr[j][i] / div) * matr[i] 

Вспомогательные функции:
1. `zero_small` - замена всех малых чисел на 0
1. `arr_print` - распечатать матрицу в красивом виде

In [57]:
def zero_small(matr):
  for i in range(matr.shape[0]):
    for j in range(matr.shape[1]):
      if is_zero(matr[i][j]):
        matr[i][j] = 0
def arr_print(matr):
  for row in matr:
    for elem in row:
      print(f"{elem} ", end="")
    print()


Функция обратного хода Гаусса, возвращает столбец решений

In [90]:
def rev_hod(matr):
  sz = matr.shape[0]
  sol = np.zeros((sz, 1))
  sol[sz - 1][0] = matr[sz - 1][sz] / matr[sz - 1][sz - 1]
  for i in range(sz - 2, -1, -1):
    sum = 0
    for j in range(i + 1, sz):
      sum += matr[i][j] * sol[j][0]
    sol[i][0] = (matr[i][sz] - sum) / matr[i][i]
  return sol

Основная программа:
1. Заполенение матрицы $A$ и столбца $f$
1. "Приклеивание" к матрице столбца
1. Выполнение прямого и обратного ходов
1. Печать вектора решений

In [91]:
A = np.ones((SIZE, SIZE))
for i in range(SIZE):
  for j in range(SIZE):
    if i != j:
      A[i][j] = 1 / (i + j + 2)
f = np.array([[1 / i for i in range(1, SIZE + 1)]]).transpose()
Asys = np.hstack((A, f))
to_diag(Asys)
#zero_small(Asys) # set all small numbers to zero (< 1e-10)
#arr_print(Asys)
sol = rev_hod(Asys)
print(sol)

[[ 9.19077109e-01]
 [ 1.75540170e-01]
 [ 6.39348240e-02]
 [ 2.72747640e-02]
 [ 1.14234685e-02]
 [ 3.51083928e-03]
 [-7.89957814e-04]
 [-3.25080145e-03]
 [-4.69787781e-03]
 [-5.55373994e-03]]


Сравнение решения с решением из `numpy`, вывод вектора разности.
Можно заметить, что различия крайне малы, что говорит о верной реализации алгоритма

In [94]:
diff  = (np.linalg.solve(A, f) - sol)
diff

array([[ 2.22044605e-16],
       [-2.77555756e-17],
       [-1.38777878e-17],
       [ 0.00000000e+00],
       [ 1.73472348e-18],
       [ 3.46944695e-18],
       [ 1.95156391e-18],
       [-4.33680869e-19],
       [ 0.00000000e+00],
       [ 0.00000000e+00]])